In [1]:
import tarfile
import urllib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,  adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
uci_tcga_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00401/"
archive_name = "TCGA-PANCAN-HiSeq-801x20531.tar.gz"

# Build the url
full_download_url = urllib.parse.urljoin(uci_tcga_url, archive_name)

# Download the file
r = urllib.request.urlretrieve (full_download_url, archive_name)

# Extract the data from the archive
tar = tarfile.open(archive_name, "r:gz")
tar.extractall()
tar.close()

In [3]:
datafile = "TCGA-PANCAN-HiSeq-801x20531/data.csv"
labels_file = "TCGA-PANCAN-HiSeq-801x20531/labels.csv"

data = np.genfromtxt(
 datafile,
 delimiter=",",
 usecols=range(1, 20532),
 skip_header=1
)

true_label_names = np.genfromtxt(
 labels_file,
 delimiter=",",
 usecols=(1,),
 skip_header=1,
 dtype="str"
)

In [4]:
from io import StringIO
df = pd.read_csv(StringIO(datafile))
dg = pd.read_csv(StringIO(labels_file))

In [5]:
df.fillna(0)

Empty DataFrame
Columns: [TCGA-PANCAN-HiSeq-801x20531/data.csv]
Index: []

In [6]:
data[:5, :3]
true_label_names[:5]


array(['PRAD', 'LUAD', 'PRAD', 'PRAD', 'BRCA'], dtype='<U4')

In [7]:
label_encoder = LabelEncoder()

true_labels = label_encoder.fit_transform(true_label_names)

true_labels[:5]

array([4, 3, 4, 4, 0], dtype=int64)

In [8]:
label_encoder.classes_

n_clusters = len(label_encoder.classes_)

In [9]:
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=42)),
    ]
 )

In [10]:
clusterer = Pipeline(
    [
        (
            "kmeans",
            KMeans(
                n_clusters=n_clusters,
                init="k-means++",
                n_init=50,
                max_iter=500,
                random_state=42,
            ),
        ),
    ]
)

In [11]:
pipe = Pipeline(
     [
         ("preprocessor", preprocessor),
         ("clusterer", clusterer)
     ]
 )

In [12]:
KKSTticks=time.perf_counter()
pipe.fit(data)
KKEDticks=time.perf_counter()
print('Kmeans所花費時間：',KKEDticks-KKSTticks)

Kmeans所花費時間： 1.3637548999995488


In [15]:
import collections

In [17]:
def purity(result,label):    # 计算纯度
        total_num = len(label)
        cluster_counter = collections.Counter(result)
        original_counter = collections.Counter(label)
        t=[]
        for k in cluster_counter:
            p_k = []
            for j in original_counter:
                 count = 0
                 for i in range(len(result)):
                     if result[i] == k and label[i] == j: # 求交集
                         count += 1
                 p_k.append(count)
            temp_t = max(p_k)
            t.append(temp_t)
        return sum(t)/total_num

KKEDticks=time.perf_counter()
print('Kmeans所花費時間：',KKEDticks-KKSTticks)
print("min_samples=1,Kmeans的純度", purity(clusterer,true_labels))

Kmeans所花費時間： 4011.8629648999977
min_samples=1,Kmeans的純度 0.0012484394506866417
